In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from time import time

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib
from sklearn import manifold, datasets

Using matplotlib backend: Qt5Agg


In [2]:

with open("instances_train.json",'r') as instances_train:
    instances_train_dict = json.load(instances_train)

categories_dict = {}
for i in instances_train_dict['categories']:
    categories_dict[i['id']]=i['name']
images=instances_train_dict['images']
annotations = instances_train_dict['annotations']

class_boxnum = {}#统计每个class的box数目
class_image={}#统计每个class所在图片
class_imagenum={}#统计每个class的图片数
for annotation in annotations:
    #统计每个class的box数目
    if categories_dict[annotation['category_id']] not in class_boxnum:
        class_boxnum[categories_dict[annotation['category_id']]] = 1
    else:
        class_boxnum[categories_dict[annotation['category_id']]] += 1
    #统计每个class所在图片
    if categories_dict[annotation['category_id']] not in class_image:
        class_image[categories_dict[annotation['category_id']]] = set()
        class_image[categories_dict[annotation['category_id']]].add(annotation['image_id'])
    else:
        class_image[categories_dict[annotation['category_id']]].add(annotation['image_id'])


for key in class_image.keys():
    class_imagenum[key]=len(class_image[key])



In [8]:
#写出类别
categories_csv = []
for i in range(80):
    #print(categories_dict[i])
    categories_csv.append({'name':categories_dict[i],'id':i})
pd.DataFrame(categories_csv).to_csv('categories_csv.csv')

In [4]:
annotations[0]

{'area': 172517.9061676068,
 'bbox': [0.1479492061, 222.2470703104, 595.7814941417, 289.56573486080003],
 'category_id': 58,
 'id': 0,
 'image_id': 23748,
 'iscrowd': 0}

In [8]:
# img_8位.jpg x1,y1,x2,y2,class_name pd.DataFrame(shop_pay_count).to_csv('shop_pay_count_demo.csv')
new_annotations=[]
for annotation in annotations:
    image_id = annotation['image_id']
    filename ='img_' + '%08d' % image_id + '.jpg'
    x1,y1,w,h = annotation['bbox']
    x2 = x1 + w
    y2 = y1 + h
    classname = categories_dict[annotation['category_id']]
    new_annotation = {'filename':filename,'x1':x1,'y1':y1,'x2':x2,'y2':y2,'classname':classname}
    new_annotations.append(new_annotation)
pd.DataFrame(new_annotations).to_csv('new_annotations.csv')
print('done')

done


In [5]:
#划分训练子集
train_id = set()
for categorie in categories_dict.values():
    if class_imagenum[categorie] < 1000:
        for i in range(int(len(class_image[categorie])*0.9)):
            train_id.add(list(class_image[categorie])[i])
    elif class_imagenum[categorie] > 1000 and  class_imagenum[categorie] < 2000:
        for i in range(int(len(class_image[categorie])*0.2)):
            train_id.add(list(class_image[categorie])[i])
    elif class_imagenum[categorie] > 2000 and  class_imagenum[categorie] < 4000:
        for i in range(int(len(class_image[categorie])*0.1)):
            train_id.add(list(class_image[categorie])[i])
    elif class_imagenum[categorie] > 4000 and  class_imagenum[categorie] < 10000:
        for i in range(int(len(class_image[categorie])*0.1)):
            train_id.add(list(class_image[categorie])[i])
    elif class_imagenum[categorie] > 10000:
        for i in range(int(len(class_image[categorie])*0.1)):
            train_id.add(list(class_image[categorie])[i])
len(train_id)

In [3]:
#划分测试集
test = []
for categorie in categories_dict.values():
    if class_imagenum[categorie] <2000:
        for i in list(class_image[categorie])[int(len(class_image[categorie])*0.9):]:
            test.append(i)
    else:
        for i in list(class_image[categorie])[-200:]:
            test.append(i)
test = set(test)
len(test)

7350

In [12]:
#生成训练子集与测试集文件
train_annotations=[]
test_annotations = []
for annotation in annotations:
    image_id = annotation['image_id']
    filename ='/disk2/zhang/ZJZSD/ZJLAB_ZSD_2019/train/img_' + '%08d' % image_id + '.jpg'
    #处理box数值
    x1,y1,w,h = annotation['bbox']
    a = lambda x : x if x > 0 else 0
    x1 = a(x1)
    y1 = a(y1)
    b = lambda x : x if x > 1 else 1
    w = b(w)
    h = b(h)
    x2 = x1 + w
    y2 = y1 + h
    x1 = round(x1)
    y1 = round(y1)
    x2 = round(x2)
    y2 = round(y2)
    
    if x1<x2 and y1<y2:
        classname = categories_dict[annotation['category_id']]
        new_annotation = {'filename':filename,'x1':x1,'y1':y1,'x2':x2,'y2':y2,'classname':classname}
        if image_id in train_id:
            train_annotations.append(new_annotation)
        elif image_id in test:
            test_annotations.append(new_annotation)
pd.DataFrame(train_annotations).to_csv('train_annotations.csv')
pd.DataFrame(test_annotations).to_csv('test_annotations.csv')
print('done')

done


In [4]:
#生成全量训练集与测试集文件
train_annotations=[]
test_annotations = []
for annotation in annotations:
    image_id = annotation['image_id']
    filename ='/disk2/zhang/ZJZSD/ZJLAB_ZSD_2019/train/img_' + '%08d' % image_id + '.jpg'
    #处理box数值
    x1,y1,w,h = annotation['bbox']
    a = lambda x : x if x > 0 else 0
    x1 = a(x1)
    y1 = a(y1)
    b = lambda x : x if x > 1 else 1
    w = b(w)
    h = b(h)
    x2 = x1 + w
    y2 = y1 + h
    x1 = round(x1)
    y1 = round(y1)
    x2 = round(x2)
    y2 = round(y2)
    
    if x1<x2 and y1<y2:
        classname = categories_dict[annotation['category_id']]
        new_annotation = {'filename':filename,'x1':x1,'y1':y1,'x2':x2,'y2':y2,'classname':classname}
        if image_id in test:
            test_annotations.append(new_annotation)
        else:
            train_annotations.append(new_annotation)
pd.DataFrame(train_annotations).to_csv('train_annotations_all.csv')
pd.DataFrame(test_annotations).to_csv('test_annotations.csv')
print('done')

done


In [2]:
#读入coco词向量进字典
coco_w2v = np.loadtxt('word_w2v.txt',dtype='float32', delimiter=',')
coco_w2v = coco_w2v.T
coco_dict={}
coco_name=
for i in range(len(coco_name)):
    coco_dict[coco_name[i]] =  coco_w2v[i]

In [17]:
#读入之江词向量到列表
with open('seen_embeddings_GloVe.json','r') as f:
    seen_embeddings_GloVe = json.load(f)
emebdd = []
label = []
for i in seen_embeddings_GloVe.items():
    key , value = i
    label.append(key)
    emebdd.append(np.array(value[0]))
    
X = np.array(emebdd)
print(label[0])

horse


In [19]:
result = np.loadtxt('自制w2v/word_w2v_l2.txt',dtype = 'float32',delimiter=',')
result = result.T
label = []
with open('oneword_list.txt','r') as f:
    for line in f.readlines():
        label.append(line)
f.close

<function TextIOWrapper.close>

In [22]:
'''t-SNE'''
t0 = time()
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
result = tsne.fit_transform(result)  # 转换后的输出
t1 = time()
print("t-SNE: %.2g sec" % (t1 - t0))  # 算法用时
def plot_embedding(data, label, title):
    x_min, x_max = np.min(data, 0), np.max(data, 0)
    data = (data - x_min) / (x_max - x_min)
 
    fig = plt.figure(figsize=(10, 10))
    #ax = plt.subplot(111)
    for i in range(data.shape[0]):
        plt.text(data[i, 0], data[i, 1], str(label[i]),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([])
    plt.yticks([])
    plt.title(title)
    return fig
fig = plot_embedding(result, label, 't-SNE embedding')
plt.show(fig)

t-SNE: 0.47 sec


In [7]:
#正则化
import keras.backend as K

def normal_w2v(v):
    v = K.l2_normalize(v,axis=0)
    v = K.eval(v)
    return v
word = np.loadtxt('word_glo.txt',dtype='float32', delimiter=',')
np.savetxt('word_glo_l2.txt',normal_w2v(word), delimiter=',')
voca = np.loadtxt('vocabulary_glo.txt',dtype='float32', delimiter=',')
np.savetxt('vocabulary_glo_l2.txt',normal_w2v(voca), delimiter=',')